# setting

In [1]:
# 필수 라이브러리 로딩
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains # 스크롤
from urllib import request # 이미지
import csv
import time

import numpy as np # 결측값 삽입용
import re # 정규표현식

In [ ]:
## 교재 테이블 리스트들 
main_title = []  # 본제
sub_title = []  # 부제
author = []  # 저자
publish = []  # 출판
date = []  # 발행일
cate = []  # 네이버 카테고리
# set_isbn = [] #set
isbn = []
tot_page = []  # 전체 쪽수
size = []  # 크기
wei = []  # 무게
img = [] # 이미지
book_info = []  # 책소개
book_review = []  # 책서평
index = [] # 목차
idx_page = [] # 목차 페이지

## isbn용 리스트들
set_isbn = [] #set
isbn_code = []
book_type = [] #제본
ver = []  # 판정보
last_ver = []  # 이전 판 정보
keyword = []  # 키워드
price = []

## 목차용 리스트들
# main_title = [] ## 본제
# isbn = []
tot_cate = []  # 교보 목차
key_cate = []
# tot_page = []

In [2]:
# 함수 정의
def page(text,ls):
    x = text.split(' ')
    try:
        if x[-1].isnumeric() == True:
            ls.append(x[-1]) # sub_p
        else:
            ls.append(np.nan)
    except:
        ls.append(np.null)

def naver():
    # 창 전환
    driver.switch_to.window(driver.window_handles[1]) 
    try: # 제목 수집
    main_title.append(driver.find_element(By.CLASS_NAME,f'bookTitle_book_name__JuBQ2').text) # 본
    sub_title.append(driver.find_element(By.CLASS_NAME,f'bookTitle_sub_title__B0uMS').text) # 부
    except:
    main_title.append(driver.find_element(By.CLASS_NAME,f'bookTitle_title_area__fspvB').text) #부제랑 안나눠졌을시
    sub_title.append(np.nan)
    
    # 저자, 출판, 발행일 수집
    info = driver.find_elements(By.CLASS_NAME, "bookTitle_inner_content__REoK1")
    author.append(info[0].text) # 저자
    publish.append(info[1].text) # 출판
    date.append(info[2].text) # 발행일

    # 책정보(카테고리, 쪽수, 무게 크기, ISBN) 수집
    sub1 = driver.find_elements(By.CLASS_NAME, "bookBasicInfo_info_detail__I0Fx5") # 카테고리, ISBN
    sub2 = driver.find_elements(By.CLASS_NAME, "bookBasicInfo_spec__qmQ_N") # 쪽수, 크기(무게)
    # cate
    cate.append(sub1[0].text)
    # code
    code = sub1[2].text
    isbn.append(code)
    # page
    tot_page.append(sub2[0].text)
    # wei
    wei.append(sub2[1].text)
    # size
    size.append(sub2[2].text)
    # image
    img_url = driver.find_element(By.XPATH, '//*[@id="container"]/div[2]/div[1]/div[1]/div[1]/div[1]/div/img').get_attribute("src")
    request.urlretrieve(img_url,f"C:\\police_img\\{isbn}.jpg")
    img.append(f'{isbn}.jpg')
    # 소개
    try:
        text = driver.find_element(By.XPATH, '//*[@id="book_section-info"]/div[3]/div[2]/div[1]').text.split('\n')
        text = [t for t in text if t != '' ]
        book_info.append(text)
    except:
        book_info.append(np.nan)
   
    # 서평
    try:
        text = driver.find_element(By.XPATH, '//*[@id="book_section-info"]/div[4]/div[2]/div[1]').text.split('\n')
        text = [t for t in text if t != '' ]
        book_review.append(text)
    except:
        book_review.append(np.nan)
        
    # 목차
    tee = driver.find_element(By.CSS_SELECTOR, '#book_section-info > div:nth-child(5) > div.infoItem_data_box__VwBf3.infoItem_expand__xBsGl > div.infoItem_data_text__bUgVI').text
    txts = tee.split('\n')
    txts = [t for t in txts if t != '']
    for t in txts:
        page(t, idx_page)
        index.append(t)
    return isbn

IndentationError: expected an indented block after 'try' statement on line 5 (2470171654.py, line 6)

In [ ]:
# 교보문고 함수
def kobo():
    driver.find_element(By.PARTIAL_LINK_TEXT, '교보').click()
    time.sleep(3)

    # tap switch
    driver.switch_to.window(driver.window_handles[2]) 

    # 원하는 작업
    cate = driver.find_element(By.CLASS_NAME, 'category_list_item').text
    tot_cate.append(cate) # 전체 카테
    key_cate.append(cate.split(' ')[-1]) # 핵심 카테
    time.sleep(2)

    driver.close() # 종료
    driver.switch_to.window(driver.window_handles[1])

- ISBN은 세트랑 세트가 아닌것이 불리되어야 하니깐, 해당페이지 들어가자 마자
resultData의 개수를 카운트해서 하나면 단일 하나 이상이면 세트 권으로 파악을 하자
re.sub(r'[^0-9]', '', string)

In [ ]:
def one_isbn():
    # 가격
    pr = driver.find_element(By.XPATH, '//*[@id="resultList_div"]/form/div/div/div/ul/li[6]').text
    price.append(pr.split(' ')[-1])
    driver.find_element(By.XPATH, f'//*[@id="resultList_div"]/form/div[{1}]/div/div/div/a').click()
    time.sleep(3)
    # 단일코드
    one = driver.find_element(By.XPATH, '//*[@id="contents"]/div/div[1]/div[2]/div[2]/ul/li[1]/div').text
    isbn_code.append(re.sub(r'[^0-9]', '', one.split(' ')[0]))
    # 제본
    book_type.append(driver.find_element(By.XPATH, '//*[@id="contents"]/div/div[1]/div[2]/div[2]/ul/li[3]/div').text)
    # 판정보
    ver.append(driver.find_element(By.XPATH, '//*[@id="contents"]/div/div[1]/div[2]/div[2]/ul/li[4]/div').text)
    # 이전판 정보
    last = driver.find_element(By.XPATH, '//*[@id="contents"]/div/div[1]/div[2]/div[2]/ul/li[5]/div').text
    last_ver.append(re.sub(r'[^0-9]', '', last.split(' ')[-1]))
    # 키워드
    keyword.append(driver.find_element(By.XPATH, '//*[@id="contents"]/div/div[1]/div[2]/div[2]/ul/li[11]/div').text)
    # 세트 코드
    set_isbn.append(np.nan)
    
    driver.close() # 종료
    driver.switch_to.window(driver.window_handles[0])  # 다시 본화면 전환


In [ ]:
def set_code():
    # 가격
    prs = driver.find_elements(By.CLASS_NAME, 'dot-list')
    for i in range(1,len(prs)+1):
        # 상품 선택
        driver.find_element(By.XPATH, f'//*[@id="resultList_div"]/form/div[{i}]/div/div/div/a').click()
        time.sleep(1)
        # 안에 정보 수 세기
        num = driver.find_elements(By.XPATH, '//*[@id="contents"]/div/div[1]/div[2]/div[2]/ul/li/div')
        if len(num) == 12:  # 개별권
            # 마지막이 안일정해서
            one = driver.find_element(By.XPATH, '//*[@id="contents"]/div/div[1]/div[2]/div[2]/ul/li[1]/div').text
            isbn_code.append(re.sub(r'[^0-9]', '', one.split(' ')[0]))
            s = driver.find_element(By.XPATH, '//*[@id="contents"]/div/div[1]/div[2]/div[2]/ul/li[2]/div').text
            set_isbn.append(re.sub(r'[^0-9]', '', s.split(' ')[0]))
            # 제본
            book_type.append(driver.find_element(By.XPATH, '//*[@id="contents"]/div/div[1]/div[2]/div[2]/ul/li[4]/div').text)
            # 판정보
            ver.append(driver.find_element(By.XPATH, '//*[@id="contents"]/div/div[1]/div[2]/div[2]/ul/li[5]/div').text)
            # 이전판 정보
            last = driver.find_element(By.XPATH, '//*[@id="contents"]/div/div[1]/div[2]/div[2]/ul/li[6]/div').text
            last_ver.append(re.sub(r'[^0-9]', '', last.split(' ')[-1]))
            # 키워드
            keyword.append(driver.find_element(By.XPATH, '//*[@id="contents"]/div/div[1]/div[2]/div[2]/ul/li[12]/div').text)
            # 가격
            price.append(np.nan) 
            time.sleep(2)
        else:  # 세트
            price.append(prs[-1].text.split(' ')[-1])
            one = driver.find_element(By.XPATH, '//*[@id="contents"]/div/div[1]/div[2]/div[2]/ul/li[1]/div').text
            set_isbn.append(re.sub(r'[^0-9]', '', one.split(' ')[0]))  # 여기선 이게 세트
            for x in [isbn_code, book_type, ver, last_ver,keyword]:
                i.append(np.nan) # 나머지 항목들은 널값처리
        driver.back()
    

In [ ]:
def search_isbn(code): 
    driver.get(f'https://www.nl.go.kr/seoji/contents/S80100000000.do?schType=simple&schStr={code}')
    num = driver.find_elements(By.CLASS_NAME, 'resultData')
    if len(num) == 1: # 단일 경우 함수
        one_isbn()
    else:
        set_code()

In [ ]:
# 상품 찾는 함수
tag = driver.find_element(By.CLASS_NAME,'style_footer__DaTQt')
# somthing element까지 스크롤
action = ActionChains(driver)

def find():
    action.move_to_element(tag).perform()
    time.sleep(2)
    elements = driver.find_elements(By.CLASS_NAME,f'bookListItem_vendor__d_xBD')
    for i,ele in enumerate(elements):
        ele.click()
        # 네이버 함수
        isbn = naver()
        # 교보문고 함수
        kobo()
        # ISBN 함수
        search_isbn(isbn)

# 페이지 넘김
n = 2
def action():
    driver.get(url=URL)
    try:
        find()
        driver.find_element(By.LINK_TEXT,f'{n}').click()
        n += 1
    except:
        pass

def isbn_func():
    driver.get(f'https://www.nl.go.kr/seoji/contents/S80100000000.do?schType=simple&schStr={code}')
    ls = [] # 결과물 담을 리스트
    # 혹시 모를 가격정보
    price = driver.find_element(By.XPATH, '//*[@id="resultList_div"]/form/div/div/div/ul/li[6]').text
    ls.append(price.split(' ')[-1])
    driver.find_element(By.XPATH, f'//*[@id="resultList_div"]/form/div[{1}]/div/div/div/a').click()
    time.sleep(3)
    ls.append(driver.find_element(By.XPATH, '//*[@id="contents"]/div/div[1]/div[2]/div[2]/ul/li[1]/div').text)
    ls.append(driver.find_element(By.XPATH, '//*[@id="contents"]/div/div[1]/div[2]/div[2]/ul/li[3]/div').text)
    ls.append(driver.find_element(By.XPATH, '//*[@id="contents"]/div/div[1]/div[2]/div[2]/ul/li[4]/div').text)
    last = driver.find_element(By.XPATH, '//*[@id="contents"]/div/div[1]/div[2]/div[2]/ul/li[5]/div').text
    ls.append(last.split('\n')[0])
    ls.append(driver.find_element(By.XPATH, '//*[@id="contents"]/div/div[1]/div[2]/div[2]/ul/li[11]/div').text)

    driver.close() # 종료
    driver.switch_to.window(driver.window_handles[0])  # 다시 본화면 전환
-  수집 내용 수정

In [ ]:
# 첫화면
URL = 'https://search.shopping.naver.com/book/search/category?bookTabType=ALL&catId=50005981&pageIndex=1&pageSize=40&sort=DATE'
ops = webdriver.ChromeOptions()
ops.add_experimental_option('excludeSwitches', ['enable-logging'])

sv = Service(executable_path= 'C:\Data\chromedriver')
driver = webdriver.Chrome(service=sv, options=ops)

driver.get(url=URL)